# Clustering Crypto

In [25]:
# Initial imports
import pandas as pd
import hvplot.pandas
from path import Path
import plotly.express as px
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
import hvplot.pandas
from bokeh.sampledata.autompg import autompg_clean as df

## Deliverable 1: Preprocessing the Data for PCA

### 1. Read in the crypto_data.csv to the Pandas DataFrame named crypto_df.

In [26]:
# Load the crypto_data.csv dataset.
file_path = "Resources/crypto_data.csv"
crypto_df = pd.read_csv(file_path, index_col=0)

### 2. Keep all the cryptocurrencies that are being traded.

In [27]:
# Keep all the cryptocurrencies that are being traded.
#crypto_df = df.copy()
crypto_df = crypto_df[crypto_df['IsTrading']== True]
print(crypto_df.shape)
crypto_df.head()

(1144, 6)


,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,True,PoW/PoS,4.199995e+01,42
365,365Coin,X11,True,PoW/PoS,NaN,2300000000
404,404Coin,Scrypt,True,PoW/PoS,1.055185e+09,532000000
611,SixEleven,SHA-256,True,PoW,NaN,611000
808,808,SHA-256,True,PoW/PoS,0.000000e+00,0


In [28]:
# Keep all the cryptocurrencies that have a working algorithm.
crypto_df[crypto_df["Algorithm"] != "N/A"]

,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,True,PoW/PoS,4.199995e+01,42
365,365Coin,X11,True,PoW/PoS,NaN,2300000000
404,404Coin,Scrypt,True,PoW/PoS,1.055185e+09,532000000
611,SixEleven,SHA-256,True,PoW,NaN,611000
808,808,SHA-256,True,PoW/PoS,0.000000e+00,0
...,...,...,...,...,...,...
SERO,Super Zero,Ethash,True,PoW,NaN,1000000000
UOS,UOS,SHA-256,True,DPoI,NaN,1000000000
BDX,Beldex,CryptoNight,True,PoW,9.802226e+08,1400222610
ZEN,Horizen,Equihash,True,PoW,7.296538e+06,21000000


### 3. Drop the `IsTrading` column.

In [29]:
# Remove the "IsTrading" column. 
crypto_df = crypto_df.drop(columns=["IsTrading"], axis=1)

### 4. Remove rows that have at least one null value.

In [30]:
# Remove rows that have at least 1 null value.
#crypto_df = crypto_df[crypto_df['TotalCoinsMined'].notna()]
crypto_df = crypto_df.dropna()
crypto_df.head()

,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,PoW/PoS,4.199995e+01,42
404,404Coin,Scrypt,PoW/PoS,1.055185e+09,532000000
808,808,SHA-256,PoW/PoS,0.000000e+00,0
1337,EliteCoin,X13,PoW/PoS,2.927942e+10,314159265359
BTC,Bitcoin,SHA-256,PoW,1.792718e+07,21000000


### 5. Filter the crypto_df DataFrame so it only has rows where coins have been mined.


In [31]:
# Keep the rows where coins are mined.
crypto_df = crypto_df[crypto_df["TotalCoinsMined"]>0]
print(crypto_df.shape)
crypto_df.head()

(532, 5)


,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,PoW/PoS,4.199995e+01,42
404,404Coin,Scrypt,PoW/PoS,1.055185e+09,532000000
1337,EliteCoin,X13,PoW/PoS,2.927942e+10,314159265359
BTC,Bitcoin,SHA-256,PoW,1.792718e+07,21000000
ETH,Ethereum,Ethash,PoW,1.076842e+08,0


In [32]:
# Check for null values
for column in crypto_df.columns:
    print(f"Column {column} has {crypto_df[column].isnull().sum()} null values")

Column CoinName has 0 null values
Column Algorithm has 0 null values
Column ProofType has 0 null values
Column TotalCoinsMined has 0 null values
Column TotalCoinSupply has 0 null values


### 6. Create a new DataFrame that holds only the cryptocurrency names, and use the crypto_df DataFrame index as the index for this new DataFrame.

In [33]:
# Create a new DataFrame that holds only the cryptocurrencies names.
#coin_name = crypto_df["CoinName"]
coin_name = crypto_df.iloc[:, [0]]
print(coin_name.shape)
coin_name.head()

(532, 1)


,CoinName
42,42 Coin
404,404Coin
1337,EliteCoin
BTC,Bitcoin
ETH,Ethereum


In [34]:
# Drop the 'CoinName' column since it's not going to be used on the clustering algorithm.
crypto_df.drop(columns=["CoinName"], axis=1, inplace=True)
crypto_df.head()

,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
42,Scrypt,PoW/PoS,4.199995e+01,42
404,Scrypt,PoW/PoS,1.055185e+09,532000000
1337,X13,PoW/PoS,2.927942e+10,314159265359
BTC,SHA-256,PoW,1.792718e+07,21000000
ETH,Ethash,PoW,1.076842e+08,0


In [35]:
# Use get_dummies() to create variables for text features.
X = pd.get_dummies(crypto_df, columns=['Algorithm','ProofType'], prefix_sep='_')
X

,TotalCoinsMined,TotalCoinSupply,Algorithm_1GB AES Pattern Search,Algorithm_536,Algorithm_Argon2d,Algorithm_BLAKE256,Algorithm_Blake,Algorithm_Blake2S,Algorithm_Blake2b,Algorithm_C11,...,ProofType_PoW/PoS,ProofType_PoW/PoS,ProofType_PoW/PoW,ProofType_PoW/nPoS,ProofType_Pos,ProofType_Proof of Authority,ProofType_Proof of Trust,ProofType_TPoS,ProofType_Zero-Knowledge Proof,ProofType_dPoW/PoW
42,4.199995e+01,42,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
404,1.055185e+09,532000000,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
1337,2.927942e+10,314159265359,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
BTC,1.792718e+07,21000000,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
ETH,1.076842e+08,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ZEPH,2.000000e+09,2000000000,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
GAP,1.493105e+07,250000000,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
BDX,9.802226e+08,1400222610,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
ZEN,7.296538e+06,21000000,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [36]:
# Standardize the data with StandardScaler
X_scaled = StandardScaler().fit_transform(X)
X_scaled[0:5]

array([[-0.11710817, -0.1528703 , -0.0433963 , -0.0433963 , -0.0433963 ,
        -0.06142951, -0.07530656, -0.0433963 , -0.06142951, -0.06142951,
        -0.0433963 , -0.0433963 , -0.19245009, -0.06142951, -0.09740465,
        -0.0433963 , -0.11547005, -0.07530656, -0.0433963 , -0.0433963 ,
        -0.15191091, -0.0433963 , -0.13118084, -0.0433963 , -0.0433963 ,
        -0.08703883, -0.0433963 , -0.0433963 , -0.0433963 , -0.0433963 ,
        -0.06142951, -0.0433963 , -0.08703883, -0.08703883, -0.08703883,
        -0.0433963 , -0.13118084, -0.13840913, -0.13840913, -0.0433963 ,
        -0.06142951, -0.0433963 , -0.07530656, -0.18168574, -0.0433963 ,
        -0.0433963 , -0.0433963 , -0.07530656, -0.15826614, -0.31491833,
        -0.0433963 , -0.08703883, -0.07530656, -0.06142951,  1.38675049,
        -0.0433963 , -0.0433963 , -0.06142951, -0.0433963 , -0.0433963 ,
        -0.0433963 , -0.0433963 , -0.0433963 , -0.0433963 , -0.0433963 ,
        -0.0433963 , -0.39879994, -0.0433963 , -0.1

## Deliverable 2: Reducing Data Dimensions Using PCA

In [37]:
# Using PCA to reduce dimension to three principal components.
pca = PCA(n_components=3)
crypto_pca = pca.fit_transform(X_scaled)
crypto_pca

array([[-0.3381678 ,  1.13233721, -0.53926664],
       [-0.32154278,  1.13320595, -0.53951358],
       [ 2.29681803,  1.88871204, -0.60939053],
       ...,
       [ 0.32291204, -2.30050668,  0.38742783],
       [-0.14431562, -2.15153964,  0.37692608],
       [-0.29245414,  0.8667366 , -0.25930849]])

In [38]:
# Create a DataFrame with the three principal components.
pcs_df = pd.DataFrame(
    crypto_pca, columns=["PC 1", "PC 2", "PC 3"],
    index=crypto_df.index
)
pcs_df.head()

,PC 1,PC 2,PC 3
42,-0.338168,1.132337,-0.539267
404,-0.321543,1.133206,-0.539514
1337,2.296818,1.888712,-0.609391
BTC,-0.144731,-1.339342,0.155707
ETH,-0.149565,-2.074024,0.368798


## Deliverable 3: Clustering Crytocurrencies Using K-Means

#### Finding the Best Value for `k` Using the Elbow Curve

In [39]:
# Find the best value for K
inertia = []
k = list(range(1, 11))

# Calculate the inertia for the range of k values
for i in k:
    km = KMeans(n_clusters=i, random_state=0)
    km.fit(pcs_df)
    inertia.append(km.inertia_)

# Create the Elbow Curve using hvPlot
elbow_data = {"k": k, "inertia": inertia}
df_elbow = pd.DataFrame(elbow_data)
df_elbow.hvplot.line(x="k", y="inertia", xticks=k, title="Elbow Curve")

:Curve   [k]   (inertia)

Running K-Means with `k=4`

In [40]:
# Initialize the K-Means model.
model = KMeans(n_clusters=4, random_state=0)

# Fit the model
model.fit(pcs_df)

# Predict clusters
predictions = model.predict(pcs_df)
predictions

array([0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1,
       1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0,
       1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1,
       1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0,
       0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1,
       0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0,
       1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0,
       0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,
       0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0,
       1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0,
       0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1,
       0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0,

In [41]:
# Create a new DataFrame including predicted clusters and cryptocurrencies features.
# Concatentate the crypto_df and pcs_df DataFrames on the same columns.
clustered_df = pd.DataFrame({
    "Algorithm" : crypto_df.Algorithm,
    "ProofType" : crypto_df.ProofType,
    "TotalCoinsMined": crypto_df.TotalCoinsMined,
    "TotalCoinSupply" : crypto_df.TotalCoinSupply,
    "PC 1": pcs_df["PC 1"],
    "PC 2": pcs_df["PC 2"],
    "PC 3": pcs_df["PC 3"],
}, index=crypto_df.index
)

#  Add a new column, "CoinName" to the clustered_df DataFrame that holds the names of the cryptocurrencies. 
clustered_df["CoinName"] = coin_name

#  Add a new column, "Class" to the clustered_df DataFrame that holds the predictions.
clustered_df["Class"] = model.labels_

# Print the shape of the clustered_df
print(clustered_df.shape)
clustered_df.head(10)

(532, 9)


,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply,PC 1,PC 2,PC 3,CoinName,Class
42,Scrypt,PoW/PoS,4.199995e+01,42,-0.338168,1.132337,-0.539267,42 Coin,0
404,Scrypt,PoW/PoS,1.055185e+09,532000000,-0.321543,1.133206,-0.539514,404Coin,0
1337,X13,PoW/PoS,2.927942e+10,314159265359,2.296818,1.888712,-0.609391,EliteCoin,0
BTC,SHA-256,PoW,1.792718e+07,21000000,-0.144731,-1.339342,0.155707,Bitcoin,1
ETH,Ethash,PoW,1.076842e+08,0,-0.149565,-2.074024,0.368798,Ethereum,1
LTC,Scrypt,PoW,6.303924e+07,84000000,-0.168906,-1.065338,-0.013027,Litecoin,1
DASH,X11,PoW/PoS,9.031294e+06,22000000,-0.397102,1.207520,-0.486440,Dash,0
XMR,CryptoNight-V7,PoW,1.720114e+07,0,-0.150639,-2.325574,0.358221,Monero,1
ETC,Ethash,PoW,1.133597e+08,210000000,-0.148011,-2.074048,0.368791,Ethereum Classic,1
ZEC,Equihash,PoW,7.383056e+06,21000000,-0.144315,-2.151540,0.376926,ZCash,1


### Deliverable 4: Visualizing Cryptocurrencies Results

#### 3D-Scatter with Clusters

In [42]:
# Creating a 3D-Scatter with the PCA data and the clusters
fig = px.scatter_3d(
    clustered_df,
    x="PC 1",
    y="PC 2",
    z="PC 3",
    color="Class",
    symbol="Class",
    width=800,
    hover_name = "CoinName",
    hover_data = ["Algorithm"]
)
fig.update_layout(legend=dict(x=0, y=1))
fig.show()

In [43]:
# Create a table with tradable cryptocurrencies.
columns = ['CoinName', 'Algorithm', 'ProofType', 'TotalCoinSupply', 'TotalCoinsMined', 'Class']
table = clustered_df.hvplot.table(columns)
table

:Table   [CoinName,Algorithm,ProofType,TotalCoinSupply,TotalCoinsMined,Class]

In [44]:
# Print the total number of tradable cryptocurrencies.
len(clustered_df)

532

In [45]:
clustered_df

,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply,PC 1,PC 2,PC 3,CoinName,Class
42,Scrypt,PoW/PoS,4.199995e+01,42,-0.338168,1.132337,-0.539267,42 Coin,0
404,Scrypt,PoW/PoS,1.055185e+09,532000000,-0.321543,1.133206,-0.539514,404Coin,0
1337,X13,PoW/PoS,2.927942e+10,314159265359,2.296818,1.888712,-0.609391,EliteCoin,0
BTC,SHA-256,PoW,1.792718e+07,21000000,-0.144731,-1.339342,0.155707,Bitcoin,1
ETH,Ethash,PoW,1.076842e+08,0,-0.149565,-2.074024,0.368798,Ethereum,1
...,...,...,...,...,...,...,...,...,...
ZEPH,SHA-256,DPoS,2.000000e+09,2000000000,2.499504,0.349239,-0.126366,ZEPHYR,0
GAP,Scrypt,PoW/PoS,1.493105e+07,250000000,-0.336218,1.132317,-0.539276,Gapcoin,0
BDX,CryptoNight,PoW,9.802226e+08,1400222610,0.322912,-2.300507,0.387428,Beldex,1
ZEN,Equihash,PoW,7.296538e+06,21000000,-0.144316,-2.151540,0.376926,Horizen,1


In [46]:
# Scaling data to create the scatter plot with tradable cryptocurrencies.

# Fit
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_df = scaler.fit_transform(clustered_df[["TotalCoinSupply", "TotalCoinsMined"]])
print(scaled_df)

[[4.20000000e-11 0.00000000e+00]
 [5.32000000e-04 1.06585544e-03]
 [3.14159265e-01 2.95755135e-02]
 ...
 [1.40022261e-03 9.90135079e-04]
 [2.10000000e-05 7.37028150e-06]
 [1.00000000e-06 1.29582282e-07]]


In [47]:
# Create a new DataFrame that has the scaled data with the clustered_df DataFrame index.
plot_df = pd.DataFrame(
    scaled_df, 
    columns=["TotalCoinSupply", "TotalCoinsMined"],
    index = clustered_df.index   
)

plot_df = pd.DataFrame({
    "TotalCoinSupply": plot_df["TotalCoinSupply"],
    "TotalCoinsMined": plot_df["TotalCoinsMined"],
    # Add the "CoinName" column from the clustered_df DataFrame to the new DataFrame. 
    "CoinName" : clustered_df["CoinName"],
    # Add the "Class" column from the clustered_df DataFrame to the new DataFrame. 
    "Class" : clustered_df["Class"],

}
)

plot_df.head(10)

,TotalCoinSupply,TotalCoinsMined,CoinName,Class
42,4.200000e-11,0.000000,42 Coin,0
404,5.320000e-04,0.001066,404Coin,0
1337,3.141593e-01,0.029576,EliteCoin,0
BTC,2.100000e-05,0.000018,Bitcoin,1
ETH,0.000000e+00,0.000109,Ethereum,1
LTC,8.400000e-05,0.000064,Litecoin,1
DASH,2.200000e-05,0.000009,Dash,0
XMR,0.000000e+00,0.000017,Monero,1
ETC,2.100000e-04,0.000115,Ethereum Classic,1
ZEC,2.100000e-05,0.000007,ZCash,1


In [52]:
# Create a hvplot.scatter plot using x="TotalCoinsMined" and y="TotalCoinSupply".
plot_df.hvplot(x='TotalCoinsMined', y='TotalCoinSupply', by='Class', kind='scatter', persist=True)


:NdOverlay   [Class]
   :Scatter   [TotalCoinsMined]   (TotalCoinSupply)